In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

SUBSET_LEN = 11
SHIFT_RECORDS = 16 # конец выборки + 6
storm_wind_speed = 10
DPI = 80
FIGSIZE, FIGSIZE_LONG = (10, 6), (12, 6)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('axes', labelsize=20)

In [21]:
source_subsets_df = pd.read_csv("source_subsets_{}.csv".format(storm_wind_speed), index_col=0)
N_subsets = len(source_subsets_df)

In [22]:
mask_4_7 = (source_subsets_df.count_storm == 4) & (source_subsets_df.count_calm == 7)
fraq_4_7 = len(source_subsets_df[mask_4_7])
mask_5_6 = (source_subsets_df.count_storm == 5) & (source_subsets_df.count_calm == 6)
fraq_5_6 = len(source_subsets_df[mask_5_6])
mask_6_5 = (source_subsets_df.count_storm == 6) & (source_subsets_df.count_calm == 5)
fraq_6_5 = len(source_subsets_df[mask_6_5])
mask_7_4 = (source_subsets_df.count_storm == 7) & (source_subsets_df.count_calm == 4)
fraq_7_4 = len(source_subsets_df[mask_7_4])

p1 = fraq_4_7 / N_subsets # вероятность распределение дней как (4, 7)
p2 = fraq_5_6 / N_subsets # вероятность распределение дней как (5, 6)
p3 = fraq_6_5 / N_subsets # вероятность распределение дней как (6, 5)
p4 = fraq_7_4 / N_subsets # вероятность распределение дней как (7, 4)

storm_days_cnt = np.array([4, 5, 6, 7])
storm_days_proba = np.array([p1, p2, p3, p4]) 

print(storm_wind_speed, fraq_4_7, fraq_5_6, fraq_6_5, fraq_7_4, N_subsets)
(fraq_4_7 + fraq_5_6 + fraq_6_5 + fraq_7_4) == N_subsets

10 130140 122492 112221 93799 458652


True

In [ ]:
def getRandomSubsetsDataframe(coordinates, dates, shift_dates=SHIFT_RECORDS, shift_coordinates=0):
    random_rows_list = []
    storms_file = open("data files/storms_{}.txt".format(storm_wind_speed), "r")
    index = 0
    for line in storms_file:
        if index % 50000 == 0:
            print(index)
        cur_storm = list(map(int, line.split()))
        i, j = cur_storm[2], cur_storm[3]
        if i + shift_coordinates >= len(coordinates) or j + shift_dates + SUBSET_LEN > len(dates):
            continue
        k = 0
        while k < SUBSET_LEN and not np.isnan(sea_df.iloc[i + shift_coordinates, j + shift_dates + k]):
            k += 1
        if k < SUBSET_LEN:
            continue
        random_rows_list.append([i + shift_coordinates, j + shift_dates])
        index += 1

    storms_file.close()
    return pd.DataFrame(random_rows_list, columns=["coordinates_index", "start_date_index"])

random_subsets_df = getRandomSubsetsDataframe(coordinates, dates)

In [ ]:
random_subsets_df

In [ ]:
def getMeanSlaDifference(index):
    i, j = random_subsets_df.iloc[index, 0], random_subsets_df.iloc[index, 1] # уже сдвинуты
    count_storm = random.choices(storm_days_cnt, storm_days_proba)[0] # count_calm = 11 - count_storm 

    random_storms = np.random.choice(np.arange(SUBSET_LEN), count_storm, replace=False)
    random_storms_indexes = np.zeros(SUBSET_LEN, dtype=bool)
    random_storms_indexes[random_storms] = True

    mean_storm_sla = sea_df.iloc[i, j:j + SUBSET_LEN][random_storms_indexes].mean()
    mean_calm_sla = sea_df.iloc[i, j:j + SUBSET_LEN][~random_storms_indexes].mean()
    return mean_storm_sla - mean_calm_sla

meanSlaDiff = np.vectorize(getMeanSlaDifference)

In [ ]:
tests_number = 1000
random_sla_diff = []
for i in range(tests_number):
    cur_sla_diff = meanSlaDiff(random_subsets_df.index)
    random_sla_diff.append(cur_sla_diff) # sla_diff для i-го теста
    print(i)

In [ ]:
random_sla_file = open("random_sla_diff_{}_{}.txt".format(storm_wind_speed, SHIFT_RECORDS), "w")
for i in range(tests_number):
    print(" ".join(map(str, random_sla_diff[i])), file=random_sla_file)
random_sla_file.close()
random_sla_diff = np.array(random_sla_diff) # shape  = (tests_number, N_subsets)

In [11]:
start_a = dict({6.7: 0.05, 10: 0.015, 15: 0.0}) # [start_a, 0.5] с шагом 0.005

In [12]:
def getProbabilityDifference(values, a_array): # P[X >= a] - P[X <= -a]
    return np.array([((values >= a).sum() - (values <= -a).sum()) / values.size for a in a_array])

def getExpectationDifference(values, cur_a, delta): # P[a < X < a + delta] - P[-a - delta < X < -a]
    p_plus_a = ((values >= cur_a) & (values <= cur_a + delta)).sum() / values.size
    p_minus_a =((values >= -cur_a - delta) & (values <= -cur_a)).sum() / values.size
    return p_plus_a - p_minus_a

def findExpectation(values, a): # sum_{a} a_i \cdot \Delta p(a)
    expectation = 0.0
    for i in range(len(a) - 1):
        expectation += a[i] * getExpectationDifference(values, a[i], a[i + 1] - a[i])
    return expectation

In [ ]:
for storm_wind in (6.7, 10, 15):
    shift = 16
    read_file_name = "data files/random_sla_diff_{}_{}.txt".format(storm_wind, shift)
    read_file = open(read_file_name, "r")
    write_random_expectations = open("data files/random_exp_{}_{}.txt".format(storm_wind, shift), "w")
    write_random_proba_diff = open("data files/random_proba_{}_{}.txt".format(storm_wind, shift), "w")

    a = np.arange(start_a[storm_wind], 0.5001, 0.005)
    for line in read_file:
        cur_sla_diff = np.array(list(map(float, line.split())))
        print(findExpectation(cur_sla_diff, a), file=write_random_expectations)
        print(" ".join(map(str, getProbabilityDifference(cur_sla_diff, a))), 
              file=write_random_proba_diff)

    read_file.close()
    write_random_expectations.close()
    write_random_proba_diff.close()
    print("Done: storm wind speed =", storm_wind)

In [19]:
abs_max_dict = dict({6.7: 1.2519796221428572, 10: 0.8886879635714285, 15: 0.6027853860714285})

In [20]:
def PrintSlaDiffHist(sla_diff, storm_wind):
    abs_max = abs_max_dict[storm_wind]
    n_bins = 200
    (y, bins, _) = plt.hist(sla_diff, bins=n_bins, density=True, range=(-abs_max, abs_max), color="xkcd:maroon")
    x = np.array([(bins[i + 1] + bins[i]) / 2 for i in range(len(bins) - 1)])
    return x, y

def GetSlaErrorExpectation(sla_diff, storm_wind):
    x, y = PrintSlaDiffHist(sla_diff, storm_wind)
    test_x = x[(x >= 0) & (x <= 0.4)]
    test_red = y[(x >= 0) & (x <= 0.4)]
    test_x_reverse = np.flip(x[(x < 0) & (x >= -0.4)])
    test_blue_reverse = np.flip(y[(x < 0) & (x >= -0.4)])

    eps = 1e-15
    mean_y = []
    for i in range(2, len(test_x) - 2):
        sla_diff_expectation = 0.0
        sum_density = 0.0
        for j in range(i - 2, i + 3):
            sla_diff_expectation += (test_red[j] - test_blue_reverse[j]) * test_x[j]
            sum_density += test_red[j] + test_blue_reverse[j]
        mean_y.append(sla_diff_expectation / sum_density)
    return mean_y

In [ ]:
shift = 16
for storm_wind in [15]:
    read_file_name = "data files/random_sla_diff_{}_{}.txt".format(storm_wind, shift)
    read_file = open(read_file_name, "r")
        
    write_sla_error = open("data files/random_sla_error_{}.txt".format(storm_wind), "w")
    cnt = 0
    for line in read_file:
        sla_diff = np.array(list(map(float, line.split())))
        sla_error = GetSlaErrorExpectation(sla_diff, storm_wind)
        print(" ".join(map(str, sla_error)), file=write_sla_error)
        cnt += 1
        print(cnt, len(sla_error))
        
    write_sla_error.close()
    read_file.close()
    print("Done: storm wind speed =", storm_wind)